# Install and import dependencies
Also loads files into python objects



In [ ]:
!pip install shapely
!pip install turfpy

  Created wheel for turfpy: filename=turfpy-0.0.7-py3-none-any.whl size=39119 sha256=f826c1fdae5613cef8ad5f22cc0f04c02282ebfe99fdf7b36dff310b8270899f
  Stored in directory: /root/.cache/pip/wheels/56/d0/d8/13858feab5280185516cd22df1adc0f75426968815b057dc5e
Successfully built turfpy


In [ ]:
import json
from turfpy import measurement
from geojson import Point, Feature, Polygon, MultiPolygon
from turfpy.measurement import boolean_point_in_polygon
import csv

In [ ]:
precincts = json.load(open('utahModifiedPrecincts.json'))

In [ ]:
votingPrecincts = json.load(open('utahVoting.json'))

# Adding Political Affiliations to Precincts
Number of republicans, democrats, others in a precinct

In [ ]:
votingDict = {}
with open('utahVoting.csv', mode='r') as cvapFile:
  next(cvapFile)
  csvReader = csv.reader(cvapFile)
  for line in csvReader:
    precinctData = {}
    precinctData['DEMOCRAT'] = float(line[5])
    precinctData['REPUBLICAN'] = float(line[4])
    precinctData['OTHER'] = float(line[6]) + float(line[7]) + float(line[8]) + float(line[9]) + float(line[10]) + float(line[11]) + float(line[12]) + float(line[13])
    votingDict[str(line[0]).upper()] = precinctData

In [ ]:
# loop through all precincts and add appropriate fields
ct = 0
for p in precincts['features']:
  name = p['properties']['GEOID20'].upper()

  # CLEANING DATA START
  if name == 'VOTING DISTRICTS NOT DEFINED':
    continue

  if name in votingDict:
    precinctData = votingDict[name]
    p['properties']['DEMOCRAT'] = precinctData['DEMOCRAT']
    p['properties']['REPUBLICAN'] = precinctData['REPUBLICAN']
    p['properties']['OTHER'] = precinctData['OTHER']
    del votingDict[name]
  else:
    print(f'{name} not found')
    ct += 1
  
print(ct)
print(len(precincts['features']))

0
2745


In [ ]:
# getting 00-000 type voting data
for p in precincts['features']:
  name = p['properties']['NAME20']
  #if 'CD116' in p['properties']:
  #  continue
  if len(name) == 6:
    for key, item in votingDict.items():
      if name in key:
        p['properties']['DEMOCRAT'] = item['DEMOCRAT']
        p['properties']['REPUBLICAN'] = item['REPUBLICAN']
        p['properties']['OTHER'] = item['OTHER']
        del votingDict[key]
        break
      # if not break early, p not found
      #print(f'{name} not found')    

In [ ]:
# not matched precincts
for p in precincts['features']:
  if 'DEMOCRAT' not in p['properties']:
    p['properties']['DEMOCRAT'] = 0
    p['properties']['REPUBLICAN'] = 0
    p['properties']['OTHER'] = 0
    print(p['properties']['NAME20'])

**Add Voting Data to CensusBlocks**

In [ ]:
censusBlocks = json.load(open('utahModifiedCensusBlocks.json'))

In [ ]:
for cB in censusBlocks['features']:
  for p in precincts['features']:
    if p['properties']['GEOID20'] == cB['properties']['PRECINCTID']:
      cB['properties']['DEMOCRAT'] = (p['properties']['DEMOCRAT'] + 0.0) / p['properties']['NUMCENSUSBLOCKS']
      cB['properties']['REPUBLICAN'] = (p['properties']['REPUBLICAN'] + 0.0) / p['properties']['NUMCENSUSBLOCKS']
      cB['properties']['OTHER'] = (p['properties']['OTHER'] + 0.0) / p['properties']['NUMCENSUSBLOCKS']

# Final Writing to Output

In [ ]:
with open('utahPreprocessedPrecincts.json', 'w') as f:
  json.dump(precincts, f, ensure_ascii=False)

with open('utahPreprocessedCensusBlocks.json', 'w') as f:
  json.dump(censusBlocks, f, ensure_ascii=False)